# <center> 流畅的Python </center>
## 第五章：一等函数
**示例5-1 创建并测试一个函数**

In [1]:
def factorial(n):
    '''returns n!'''
    return 1 if n < 2 else n * factorial(n-1)
print(factorial(42))
print(factorial.__doc__)
print(type(factorial))


1405006117752879898543142606244511569936384000000000
returns n!
<class 'function'>


**示例5-2 通过别的名称使用函数，再把函数作为参数传递**

In [2]:
fact = factorial
print(fact)
print(fact(5))
print(map(factorial, range(11)))
print(list(map(fact, range(11))))

<function factorial at 0x105726ea0>
120
[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]


**示例5-3 根据单词长度给一个列表排序**

In [4]:
fruits = ['strawberry', 'fig', 'apple', 'cherry', 'raspberry', 'banana']
print(sorted(fruits, key=len))

['fig', 'apple', 'cherry', 'banana', 'raspberry', 'strawberry']


**示例5-4 根据反向拼写给一个单词列表排序**

In [5]:
def reverse(word):
    return word[::-1]
print(reverse('testing'))
print(sorted(fruits, key=reverse))

gnitset
['banana', 'apple', 'fig', 'raspberry', 'strawberry', 'cherry']


**示例5-5 计算阶乘列表，map和filter与列表推导比较**

In [6]:
print(list(map(fact, range(6))))
print([fact(n) for n in range(6)])
print(list(map(factorial, filter(lambda n: n % 2, range(6)))))
print([factorial(n) for n in range(6) if n % 2])

[1, 1, 2, 6, 24, 120]
[1, 1, 2, 6, 24, 120]
[1, 6, 120]
[1, 6, 120]


**示例5-6 使用reduce和sum计算0-99之和**

In [7]:
from functools import reduce
from operator import add
print(reduce(add, range(100)))
print(sum(range(100)))

4950
4950


**示例5-7 使用lambda表达式反转拼写，然后以此给单词列表排序**

In [8]:
print(sorted(fruits, key=lambda word: word[::-1]))


['banana', 'apple', 'fig', 'raspberry', 'strawberry', 'cherry']


**可调用对象**

In [9]:
print([callable(obj) for obj in (abs, str, 13)])

[True, True, False]


**示例5-8 调用BingoCage实例，从打乱的列表中取出一个元素**

In [10]:
import random
class BingoCage:
    def __init__(self, items):
        self._items = list(items)
        random.shuffle(self._items)
    
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
    
    def __call__(self):
        return self.pick()

bingo = BingoCage(range(3))
print(bingo.pick())
print(bingo())
print(callable(bingo))

2
1
True


**示例5-9 列出常规对象没有而函数有的属性**

In [11]:
class C: pass
obj = C()
def func(): pass
print(sorted(set(dir(func)) - set(dir(obj))))

['__annotations__', '__call__', '__closure__', '__code__', '__defaults__', '__get__', '__globals__', '__kwdefaults__', '__name__', '__qualname__']


**示例5-10 tag函数用于生成HTML标签，使用名为cls的关键字参数传入"class"属性，这是一种变通方法，因为“class”是Python的关键字**

In [12]:
def tag(name, *content, cls=None, **attrs):
    """生成一个或多个HTML标签"""
    if cls is not None:
        attrs['class'] = cls
    if attrs:
        attr_str = ''.join(' %s="%s"' % (attr, value) for attr, value in sorted(attrs.items()))
    else:
        attr_str = ''
    if content:
        return '\n'.join('<%s%s>%s</%s>' % (name, attr_str, c, name) for c in content)
    else:
        return '<%s%s />' % (name, attr_str)
    

**示例5-11 tag函数众多调用方式中的几种**

In [13]:
print(tag('br'))
print(tag('p', 'hello'))
print(tag('p', 'hello', 'world'))
print(tag('p', 'hello', id=33))
print(tag('p', 'hello', 'world', cls='sidebar'), '\n')
print(tag(content='testing', name='img'))
my_tag = {'name': 'img', 'title': 'Sunset Boulevard', 'src': 'sunset.jpg', 'cls': 'framed'}
print(tag(**my_tag))

<br />
<p>hello</p>
<p>hello</p>
<p>world</p>
<p id="33">hello</p>
<p class="sidebar">hello</p>
<p class="sidebar">world</p> 

<img content="testing" />
<img class="framed" src="sunset.jpg" title="Sunset Boulevard" />


**仅限关键字参数**

In [14]:
def f(a, *, b):
    return (a, b)

print(f(1, b=2))
print(f(1, 2))

(1, 2)


TypeError: f() takes 1 positional argument but 2 were given

**示例5-15 在指定长度附近截断字符串的函数**

In [16]:
def clip(text, max_len=80):
    """在max_len前面或者后面的第一个空格处截断文本"""
    end = None
    if(len(text) > max_len):
        space_before = text.rfind(' ', 0, max_len)
        if(space_before > 0):
            end = space_before
        else:
            space_after = text.rfind(' ', max_len)
            if(space_after >= 0):
                end = space_after
    if end is None:
        end = len(text)
    return text[:end].rstrip()

**示例5-16 提取关于函数参数的信息**

In [18]:
print(clip.__defaults__)
print(clip.__code__)
print(clip.__code__.co_varnames)
print(clip.__code__.co_argcount)


(80,)
<code object clip at 0x103bfe540, file "<ipython-input-16-bd88deec75de>", line 1>
('text', 'max_len', 'end', 'space_before', 'space_after')
2


**示例5-17 提取函数的签名**

In [19]:
from inspect import signature
sig = signature(clip)
print(sig)
print(str(sig))
for name, param in sig.parameters.items():
    print(param.kind, ':', name, '=', param.default)


(text, max_len=80)
(text, max_len=80)
POSITIONAL_OR_KEYWORD : text = <class 'inspect._empty'>
POSITIONAL_OR_KEYWORD : max_len = 80


**示例5-18 把tag函数的签名绑定到一个参数字典上**

In [20]:
import inspect
sig = inspect.signature(tag)
my_tag = {'name': 'img', 'title': 'Sunset Boulevard', 'src': 'sunset.jpg', 'cls': 'framed'}
bound_args = sig.bind(**my_tag)
print(bound_args)
for name, value in bound_args.arguments.items():
    print(name, '=', value)
del my_tag['name']
bound_args = sig.bind(**my_tag)

<BoundArguments (name='img', cls='framed', attrs={'title': 'Sunset Boulevard', 'src': 'sunset.jpg'})>
name = img
cls = framed
attrs = {'title': 'Sunset Boulevard', 'src': 'sunset.jpg'}


TypeError: missing a required argument: 'name'

**示例5-19 有注解的clip函数**

In [21]:
def clip(text:str, max_len:'int > 0'=80) -> str:
    """在max_len前面或者后面的第一个空格处截断文本"""
    end = None
    if(len(text) > max_len):
        space_before = text.rfind(' ', 0, max_len)
        if(space_before > 0):
            end = space_before
        else:
            space_after = text.rfind(' ', max_len)
            if(space_after >= 0):
                end = space_after
    if end is None:
        end = len(text)
    return text[:end].rstrip()

print(clip.__annotations__)

{'text': <class 'str'>, 'max_len': 'int > 0', 'return': <class 'str'>}


**示例5-20 从函数签名中提取注释**

In [23]:
from inspect import signature
sig = signature(clip)
print(sig.return_annotation)
for param in sig.parameters.values():
    note = repr(param.annotation).ljust(13)
    print(note, ':', param.name, '=', param.default)

<class 'str'>
<class 'str'> : text = <class 'inspect._empty'>
'int > 0'     : max_len = 80


**示例5-22 使用reduce和operator.mul函数计算阶乘**

In [24]:
from functools import reduce
from operator import mul
def fact(n):
    return reduce(mul, range(1, n+1))
print(fact(3))

6


**示例5-23 演示使用itemgetter排序一个元组列表**

In [26]:
metro_data = [('Tokyo', 'JP', 36.933, (35.689722, 139.691667)), 
               ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)), 
               ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)), 
               ('New York-Newark', 'US', 20.104, (40.808611, -74.020386)), 
               ('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833)), 
              ]

from operator import itemgetter
for city in sorted(metro_data, key=itemgetter(1)):
    print(city)
print('\n')

cc_name = itemgetter(1, 0)
for city in metro_data:
    print(cc_name(city))
    

('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833))
('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889))
('Tokyo', 'JP', 36.933, (35.689722, 139.691667))
('Mexico City', 'MX', 20.142, (19.433333, -99.133333))
('New York-Newark', 'US', 20.104, (40.808611, -74.020386))


('JP', 'Tokyo')
('IN', 'Delhi NCR')
('MX', 'Mexico City')
('US', 'New York-Newark')
('BR', 'Sao Paulo')


**示例5-24 定义一个namedtuple， 名为metro_data， 演示使用attrgetter处理它**

In [27]:
from collections import namedtuple
LatLong = namedtuple('LatLong', 'lat long')
Metropoils = namedtuple('Metropoils', 'name cc pop coord')
metro_areas = [Metropoils(name, cc, pop, LatLong(lat, long))
              for name, cc, pop, (lat, long) in metro_data]
print(metro_areas[0])
print(metro_areas[0].coord.lat)

from operator import attrgetter
name_lat = attrgetter('name', 'coord.lat')
for city in sorted(metro_areas, key=attrgetter('coord.lat')):
    print(name_lat(city))

Metropoils(name='Tokyo', cc='JP', pop=36.933, coord=LatLong(lat=35.689722, long=139.691667))
35.689722
('Sao Paulo', -23.547778)
('Mexico City', 19.433333)
('Delhi NCR', 28.613889)
('Tokyo', 35.689722)
('New York-Newark', 40.808611)


**示例5-25 methodcaller使用示例：第二个测试展示绑定额外参数的方式**

In [28]:
from operator import methodcaller
s = 'The time has come'
upcase = methodcaller('upper')
print(upcase(s))
hiphenare = methodcaller('replace', ' ', '-')
print(hiphenare(s))

THE TIME HAS COME
The-time-has-come


**示例5-26 使用partial把一个两参数函数改编成需要单参数的可调用对象**

In [29]:
from operator import mul
from functools import partial
triple = partial(mul, 3)
print(triple(7))
print(list(map(triple, range(1, 10))))

21
[3, 6, 9, 12, 15, 18, 21, 24, 27]


**示例5-27 使用partial建立一个便利的Unicode规范化函数**

In [30]:
import unicodedata, functools
nfc = functools.partial(unicodedata.normalize, 'NFC')
s1 = 'café'
s2 = 'cafe\u0301'
print((s1, s2))
print(s1 == s2)
print(nfc(s1) == nfc(s2))

('café', 'café')
False
True


**示例5-28 把partial应用到示例5-10中定义的tag函数上**

In [31]:
print(tag)
picture = partial(tag, 'img', cls='pic-frame')
print(picture(src='wumpus.jepg'))
print(picture)
print(picture.func)
print(picture.args)
print(picture.keywords)

<function tag at 0x10f4f0c80>
<img class="pic-frame" src="wumpus.jepg" />
functools.partial(<function tag at 0x10f4f0c80>, 'img', cls='pic-frame')
<function tag at 0x10f4f0c80>
('img',)
{'cls': 'pic-frame'}
